## Data Ingestion

In [1]:
import pandas as pd

column_names = ['id', 'text', 'label']
twitter_15_train = pd.read_csv("/kaggle/input/twitter15-16/twitter15.train", sep="\t", 
                               header=None, names=column_names)
twitter_15_test = pd.read_csv("/kaggle/input/twitter15-16/twitter15.test", sep="\t", 
                              header=None, names=column_names)
twitter_15_dev = pd.read_csv("/kaggle/input/twitter15-16/twitter15.dev", sep="\t", 
                             header=None, names=column_names)

In [2]:
twitter_15_train.head()

,id,text,label
0,724703995147751424,"american family association gets 500,000 to si...",unverified
1,358591089462099968,this week's top story: george zimmerman wins f...,false
2,775672628493357057,clinton hides failing health? full disclosure ...,unverified
3,364589696573124609,fukushima: highly radioactive water seeping in...,false
4,549927969032916993,a transgender 17-year old left a suicide note ...,unverified


In [3]:
twitter_15_train.shape, twitter_15_test.shape, twitter_15_dev.shape

((1005, 3), (336, 3), (149, 3))

In [4]:
twitter_16_train = pd.read_csv("/kaggle/input/twitter15-16/twitter16.train", sep="\t", 
                               header=None, names=column_names)
twitter_16_test = pd.read_csv("/kaggle/input/twitter15-16/twitter16.test", sep="\t", 
                              header=None, names=column_names)
twitter_16_dev = pd.read_csv("/kaggle/input/twitter15-16/twitter16.dev", sep="\t", 
                             header=None, names=column_names)

In [5]:
twitter_16_train.head()

,id,text,label
0,692929779696275456,ohio lawmakers want to know why state’s epa di...,non-rumor
1,693858804279201794,poor women in india are fighting for the right...,non-rumor
2,693648684857323521,spoiler alert: leo and kate were ridiculously ...,non-rumor
3,620367840902782976,translucent butterfly - beautiful! ' URL,false
4,693939356390653952,michael oher got a text from cam newton during...,non-rumor


In [6]:
twitter_16_train.shape, twitter_16_test.shape, twitter_16_dev.shape

((552, 3), (184, 3), (82, 3))

In [7]:
twitter_train = pd.concat([twitter_15_train, twitter_16_train], ignore_index=True, axis=0)
twitter_test = pd.concat([twitter_15_test, twitter_16_test], ignore_index=True, axis=0)
twitter_dev = pd.concat([twitter_15_dev, twitter_15_test], ignore_index=True, axis=0)

In [8]:
twitter_train.shape, twitter_test.shape, twitter_dev.shape

((1557, 3), (520, 3), (485, 3))

## Data Preparation

In [9]:
LABELS = twitter_train['label'].unique().tolist()
train_label = twitter_train['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/3616623573.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = twitter_train['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [10]:
train_data = twitter_train['text'].tolist()

In [11]:
dev_data = twitter_dev['text'].tolist()
dev_label = twitter_dev['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/3987848631.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = twitter_dev['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [12]:
test_data = twitter_test['text'].tolist()
test_label = twitter_test['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/2548705681.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = twitter_test['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [13]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [14]:
from transformers import BartTokenizer, BartModel, AdamW

max_length = 128

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [16]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [17]:
num_classes = 4

In [29]:
class BARTClassifier(nn.Module):
    def __init__(self, bart_model, num_classes, dropout=0.1):
        super(BARTClassifier, self).__init__()
        self.bart = bart_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bart(input_ids=input_ids, attention_mask=attention_mask)
        
        last_hidden_state = outputs.last_hidden_state  
        
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_hidden_states = torch.sum(last_hidden_state * attention_mask_expanded, 1)
        sum_attention_mask = torch.clamp(attention_mask_expanded.sum(1), min=1e-9)
        pooled_output = sum_hidden_states / sum_attention_mask
        
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [30]:
bart_model = BartModel.from_pretrained('facebook/bart-base')

In [31]:
model = BARTClassifier(bart_model, num_classes)

In [32]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BARTClassifier(
  (bart): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Layer

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'f1': f1_score(y_true, y_pred, average='weighted')
    }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [34]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [35]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [36]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [37]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 49/49 [00:22<00:00,  2.22it/s]



Epoch 1/5
Training Metrics:
Loss: 1.3080
Accuracy: 0.3809
Precision: 0.3809
Recall: 0.3809
F1: 0.3769
Roc_auc: 0.6439

Validation Metrics:
Accuracy: 0.4969
Precision: 0.5135
Recall: 0.4969
F1: 0.4654
Roc_auc: 0.7754


Epoch 2/5: 100%|██████████| 49/49 [00:21<00:00,  2.28it/s]



Epoch 2/5
Training Metrics:
Loss: 1.0010
Accuracy: 0.5902
Precision: 0.5881
Recall: 0.5902
F1: 0.5881
Roc_auc: 0.8266

Validation Metrics:
Accuracy: 0.6351
Precision: 0.6919
Recall: 0.6351
F1: 0.6351
Roc_auc: 0.8784


Epoch 3/5: 100%|██████████| 49/49 [00:21<00:00,  2.28it/s]



Epoch 3/5
Training Metrics:
Loss: 0.7005
Accuracy: 0.7270
Precision: 0.7266
Recall: 0.7270
F1: 0.7263
Roc_auc: 0.9203

Validation Metrics:
Accuracy: 0.7175
Precision: 0.7436
Recall: 0.7175
F1: 0.7187
Roc_auc: 0.9143


Epoch 4/5: 100%|██████████| 49/49 [00:21<00:00,  2.27it/s]



Epoch 4/5
Training Metrics:
Loss: 0.4548
Accuracy: 0.8362
Precision: 0.8366
Recall: 0.8362
F1: 0.8358
Roc_auc: 0.9676

Validation Metrics:
Accuracy: 0.7258
Precision: 0.7446
Recall: 0.7258
F1: 0.7266
Roc_auc: 0.9225


Epoch 5/5: 100%|██████████| 49/49 [00:21<00:00,  2.28it/s]



Epoch 5/5
Training Metrics:
Loss: 0.3295
Accuracy: 0.8799
Precision: 0.8801
Recall: 0.8799
F1: 0.8799
Roc_auc: 0.9829

Validation Metrics:
Accuracy: 0.7753
Precision: 0.7789
Recall: 0.7753
F1: 0.7753
Roc_auc: 0.9363


In [38]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,1.307973,0.380861,0.380927,0.380861,0.376916,0.643918
1,1.001047,0.590238,0.588150,0.590238,0.588135,0.826602
2,0.700464,0.727039,0.726627,0.727039,0.726290,0.920278
3,0.454798,0.836224,0.836590,0.836224,0.835846,0.967561
4,0.329534,0.879897,0.880096,0.879897,0.879890,0.982924


In [39]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.496907,0.513509,0.496907,0.465432,0.775396
1,0.635052,0.691921,0.635052,0.635077,0.878434
2,0.717526,0.743569,0.717526,0.718711,0.914341
3,0.725773,0.744601,0.725773,0.726576,0.922476
4,0.775258,0.778934,0.775258,0.775331,0.936322


In [40]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.7788461538461539,
 'precision': 0.7815475919499817,
 'recall': 0.7788461538461539,
 'f1': 0.7789192944775003,
 'roc_auc': 0.9371115846663921}